In [1]:
!pip install gradio crewai langchain-groq duckduckgo-search python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.2/191.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.7/463.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import gradio as gr
from crewai import Agent, Task, Crew, Process
from langchain.chat_models import ChatLiteLLM
from langchain.tools import DuckDuckGoSearchRun
import json
from datetime import datetime
import logging
import traceback
from litellm import completion

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the LLM and search tool
try:
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # Try environment variable first
    if not GROQ_API_KEY:
        from google.colab import userdata
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
        os.environ["GROQ_API_KEY"] = GROQ_API_KEY

    logger.info("Successfully loaded GROQ API key")

    # Initialize LLM using litellm wrapper
    groq_llm = ChatLiteLLM(
        model_name="groq/mixtral-8x7b-32768",
        temperature=0.3,
        max_tokens=32768,
        api_key=GROQ_API_KEY
    )
    logger.info("Successfully initialized Groq LLM")

    search_tool = DuckDuckGoSearchRun()
    logger.info("Successfully initialized DuckDuckGo search tool")

except Exception as e:
    logger.error(f"Initialization error: {str(e)}")
    logger.error(traceback.format_exc())
    raise e

class FoodSafetySystem:
    def __init__(self):
        try:
            logger.info("Initializing FoodSafetySystem agents...")

            self.haccp_monitor = Agent(
                role='HACCP Monitor',
                goal='Monitor critical control points and food safety procedures',
                backstory='Expert in HACCP principles and food safety monitoring',
                llm=groq_llm,
                tools=[search_tool],
                verbose=True
            )

            self.quality_inspector = Agent(
                role='Quality Inspector',
                goal='Assess food quality parameters and safety standards',
                backstory='Specialist in food quality control and safety standards',
                llm=groq_llm,
                tools=[search_tool],
                verbose=True
            )

            self.compliance_auditor = Agent(
                role='Compliance Auditor',
                goal='Audit documentation and regulatory compliance',
                backstory='Expert in food safety regulations and compliance auditing',
                llm=groq_llm,
                tools=[search_tool],
                verbose=True
            )

            self.risk_predictor = Agent(
                role='Risk Predictor',
                goal='Predict potential food safety risks and violations',
                backstory='Specialist in predictive analytics for food safety',
                llm=groq_llm,
                tools=[search_tool],
                verbose=True
            )

            logger.info("Successfully initialized all agents")

        except Exception as e:
            logger.error(f"Error initializing agents: {str(e)}")
            logger.error(traceback.format_exc())
            raise e

    def run_safety_assessment(self, documents: list) -> list:
        try:
            logger.info("Starting safety assessment")
            logger.info(f"Input documents: {documents}")

            tasks = [
                Task(
                    description=f"Analyze HACCP procedures in: {documents}",
                    agent=self.haccp_monitor,
                    expected_output="A detailed analysis of HACCP procedures including critical control points, monitoring procedures, and any deviations identified."
                ),
                Task(
                    description=f"Assess quality control in: {documents}",
                    agent=self.quality_inspector,
                    expected_output="A comprehensive assessment of quality control measures including test results, product specifications, and quality metrics."
                ),
                Task(
                    description=f"Audit compliance in: {documents}",
                    agent=self.compliance_auditor,
                    expected_output="A detailed compliance audit report highlighting regulatory requirements, compliance status, and any identified gaps."
                ),
                Task(
                    description=f"Predict risks in: {documents}",
                    agent=self.risk_predictor,
                    expected_output="A risk assessment report identifying potential food safety hazards, their likelihood, and recommended preventive measures."
                )
            ]

            crew = Crew(
                agents=[
                    self.haccp_monitor,
                    self.quality_inspector,
                    self.compliance_auditor,
                    self.risk_predictor
                ],
                tasks=tasks,
                process=Process.sequential,
                verbose=True  # Increased verbosity for debugging
            )

            logger.info("Starting crew kickoff")
            results = crew.kickoff()
            logger.info(f"Raw results from crew: {results}")

            # Ensure results are properly formatted
            formatted_results = []
            for result in results:
                if result is None:
                    formatted_results.append("No results available")
                else:
                    formatted_results.append(str(result))

            return formatted_results

        except Exception as e:
            logger.error(f"Error in safety assessment: {str(e)}")
            logger.error(traceback.format_exc())
            raise e

def process_food_safety(
    production_docs: str,
    haccp_plan: str,
    quality_records: str,
    progress=gr.Progress()
) -> tuple:
    try:
        logger.info("Starting food safety processing")
        progress(0.1, desc="Initializing...")

        # Validate inputs
        if not all([production_docs.strip(), haccp_plan.strip(), quality_records.strip()]):
            raise ValueError("All input fields must be filled out")

        documents = [
            f"Production Documentation:\n{production_docs}",
            f"HACCP Plan:\n{haccp_plan}",
            f"Quality Control Records:\n{quality_records}"
        ]

        progress(0.2, desc="Running analysis...")
        safety_system = FoodSafetySystem()

        # Run analysis with explicit error handling
        try:
            results = safety_system.run_safety_assessment(documents)
            logger.info(f"Analysis results: {results}")
        except Exception as e:
            logger.error(f"Analysis failed: {str(e)}")
            raise Exception(f"Analysis failed: {str(e)}")

        progress(0.8, desc="Processing results...")

        # Format results with detailed error checking
        if not results or len(results) < 4:
            raise ValueError("Incomplete results received from analysis")

        formatted_results = [str(r) if r is not None else "No results available" for r in results]

        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"food_safety_report_{timestamp}.json"

        with open(filename, 'w') as f:
            json.dump({
                "haccp_monitoring": formatted_results[0],
                "quality_inspection": formatted_results[1],
                "compliance_audit": formatted_results[2],
                "risk_prediction": formatted_results[3]
            }, f, indent=2)

        progress(1.0, desc="Complete!")
        return (*formatted_results, f"Report saved as: {filename}")

    except Exception as e:
        logger.error(f"Error in process_food_safety: {str(e)}")
        logger.error(traceback.format_exc())
        detailed_error = f"Detailed error: {str(e)}\n{traceback.format_exc()}"
        return (detailed_error,) * 5

# Create the Gradio interface
with gr.Blocks(title="Food Safety Compliance System") as interface:
    gr.Markdown("# AI-Powered Food Safety Compliance System")

    with gr.Row():
        with gr.Column():
            production_docs = gr.Textbox(
                label="Production Documentation",
                placeholder="Enter production details...",
                lines=8,
                value=""
            )
            haccp_plan = gr.Textbox(
                label="HACCP Plan",
                placeholder="Enter HACCP plan...",
                lines=8,
                value=""
            )
            quality_records = gr.Textbox(
                label="Quality Control Records",
                placeholder="Enter quality records...",
                lines=8,
                value=""
            )
            submit_btn = gr.Button("Run Analysis", variant="primary")

    with gr.Row():
        with gr.Column():
            haccp_output = gr.Textbox(
                label="HACCP Monitoring Results",
                value="Awaiting analysis...",
                lines=5
            )
            quality_output = gr.Textbox(
                label="Quality Inspection Results",
                value="Awaiting analysis...",
                lines=5
            )
            compliance_output = gr.Textbox(
                label="Compliance Audit Results",
                value="Awaiting analysis...",
                lines=5
            )
            risk_output = gr.Textbox(
                label="Risk Prediction Results",
                value="Awaiting analysis...",
                lines=5
            )
            file_output = gr.Textbox(
                label="Report Status",
                value="Awaiting analysis..."
            )

    submit_btn.click(
        fn=process_food_safety,
        inputs=[production_docs, haccp_plan, quality_records],
        outputs=[haccp_output, quality_output, compliance_output, risk_output, file_output]
    )

# Launch with debugging enabled
interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d657032fd5126e45af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Agent: HACCP Monitor
## Task: Analyze HACCP procedures in: ['Production Documentation:\nfacilityName: "Fresh Dairy Solutions Inc.",\n      productLine: "Pasteurized Whole Milk",\n      batchNumber: "ML-2024-0428-01",\n      productionDate: "2024-04-28",\n      productionDetails: `\n        - Production Start Time: 06:00\n        - Batch Size: 5000 liters\n        - Pasteurization Temperature: 72.5°C\n        - Holding Time: 15 seconds\n        - Cooling Temperature: 4°C\n        - Equipment: Pasteurizer Unit #2\n        - Operator: J. Smith\n        - Supervisor: M. Johnson', 'HACCP Plan:\n      Critical Control Points (CCPs):\n      1. Pasteurization (CCP-1)\n         - Critical Limit: Min 72°C for 15 seconds\n         - Monitoring: Continuous temperature recording\n         - Corrective Action: If temperature drops below 72°C, divert flow and reprocess\n         \n      2. Cooling (CCP-2)\n         - Critical Limit: Max 4°C within 2 hours\n         - Monitoring: Temperature check e

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1748: UserWarning: A function (process_food_safety) returned too many output values (needed: 5, returned: 6). Ignoring extra values.
    Output components:
        [textbox, textbox, textbox, textbox, textbox]
    Output values returned:
        ["('raw', 'Risk Assessment Report:\n\n1. CCP-1: Pasteurization\nPotential Risk: Inadequate pasteurization temperature or holding time.\nLikelihood: Low, as the production documentation shows compliance with the regulatory requirement (72°C for 15 seconds).\nRecommended Preventive Measures: Regularly monitor and calibrate pasteurization equipment to ensure consistent temperature and time.\n\n2. CCP-2: Cooling\nPotential Risk: Microbial contamination due to improper cooling.\nLikelihood: Unknown, as the time taken for cooling is not documented.\nRecommended Preventive Measures: Implement a system to record cooling time, and ensure cooling occurs within 2 hours. Regularly monitor and documen